In [2]:
import requests
from bs4 import BeautifulSoup as BS
from IPython.core.display import HTML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## CLEANING IMDb_Movies 

#### Read in IMDb_Movies Table

In [7]:
movie_df = pd.read_csv("../Data/Old_Dirty_Data/IMDb_Movies.csv")

In [31]:
movie_df.head(10)

,Title_ID,Title,Director,Rating,Genre,Runtime,Release_Year,Award_Noms,Award_Wins,Score_ID
0,17761,The Shawshank Redemption,Frank Darabont,R,Drama,142,1994,43,21,7041
1,17762,The Godfather,Francis Ford Coppola,R,"Crime, Drama",175,1972,30,32,7042
2,17763,The Dark Knight,Christopher Nolan,PG-13,"Action, Crime, Drama",152,2008,163,160,7043
3,17764,The Godfather Part II,Francis Ford Coppola,R,"Crime, Drama",202,1974,21,17,7044
4,17765,12 Angry Men,Sidney Lumet,Approved,"Crime, Drama",96,1957,13,17,7045
5,17766,Schindler's List,Steven Spielberg,R,"Biography, Drama, History",195,1993,49,91,7046
6,17767,The Lord of the Rings: The Return of the King,Peter Jackson,PG-13,"Action, Adventure, Drama",201,2003,124,209,7047
7,17768,Pulp Fiction,Quentin Tarantino,R,"Crime, Drama",154,1994,75,70,7048
8,17769,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,PG-13,"Action, Adventure, Drama",178,2001,126,121,7049
9,177610,"The Good, the Bad and the Ugly",Sergio Leone,R,"Adventure, Western",178,1966,6,3,70410


In [30]:
movie_df[50:60]

,Title_ID,Title,Director,Rating,Genre,Runtime,Release_Year,Award_Noms,Award_Wins,Score_ID
50,177651,Alien,Ridley Scott,R,"Horror, Sci-, Fi",117,1979,22,18,70451
51,177652,City Lights,Charles Chaplin,G,"Comedy, Drama, Romance",87,1931,1,4,70452
52,177653,Apocalypse Now,Francis Ford Coppola,R,"Drama, Mystery, War",147,1979,33,21,70453
53,177654,Memento,Christopher Nolan,R,"Mystery, Thriller",113,2000,60,57,70454
54,177655,Raiders of the Lost Ark,Steven Spielberg,PG,"Action, Adventure",115,1981,24,38,70455
55,177656,Django Unchained,Quentin Tarantino,R,"Drama, Western",165,2012,158,58,70456
56,177657,WALL·E,Andrew Stanton,G,"Animation, Adventure, Family",98,2008,95,95,70457
57,177658,The Lives of Others,Florian Henckel von Donnersmarck,R,"Drama, Mystery, Thriller",137,2006,38,80,70458
58,177659,Sunset Blvd.,Billy Wilder,Passed,"Drama, Film-, Noir",110,1950,20,19,70459
59,177660,Paths of Glory,Stanley Kubrick,Approved,"Drama, War",88,1957,4,7,70460


##### Drop 'Unnamed: 0' which I believe is the index in the csv being imported in

In [13]:
movie_df.pop("Unnamed: 0")

0        0
1        1
2        2
3        3
4        4
      ... 
245    245
246    246
247    247
248    248
249    249
Name: Unnamed: 0, Length: 250, dtype: int64

In [15]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title_ID      250 non-null    int64 
 1   Title         250 non-null    object
 2   Director      250 non-null    object
 3   Rating        250 non-null    object
 4   Genre         250 non-null    object
 5   Runtime       250 non-null    int64 
 6   Release_Year  250 non-null    int64 
 7   Award_Noms    250 non-null    int64 
 8   Award_Wins    250 non-null    int64 
 9   Score_ID      250 non-null    int64 
dtypes: int64(6), object(4)
memory usage: 19.7+ KB


#### Next, I need to clean up Ratings and the discrepencies there

Source: Motion Picture Association (MPA)
Link: https://www.motionpictures.org/who-we-are/#our-history
Summary: Founded in 1922, MPA started with the Hays Code system which which simply approved or disapproved based on whether the movies were deemed "moral" or "immoral". In 1968, the Hayes Code was replaced with the modern rating system we have today.
    
Source: IMDb
Link: https://help.imdb.com/article/contribution/titles/certificates/GU757M8ZJ9ZPXB39#
Summary: This website goes through each country and explains the ratings 

From the research that I have conducted, I have decided that this will be the standard to limit the discrepencies in the rating categories.<br>
<bold> DECISION: <bold> Passed and Approved will be combined to be "Approved". Furthermore, Unrated and Not Rated show very little difference in the research and typically does not follow any standard for why one is used over another; thus, these will be combined as "Not Rated"

In [22]:
movie_df['Rating'].value_counts()

R                        101
PG                        37
PG-13                     34
Not Rated                 24
G                         19
Passed                    16
Approved                  13
Not Available on IMDb      1
TV-PG                      1
Unrated                    1
X                          1
TV-MA                      1
GP                         1
Name: Rating, dtype: int64

## CLEANING IMDb_Score

#### Read in IMDb_Score Table

In [11]:
score_df = pd.read_csv("../Data/Old_Dirty_Data/IMDb_Score.csv")

In [12]:
score_df.head()

,Unnamed: 0,Score_ID,IMDb_Score,Audience_Reviews,US_Users,Non_US_Users,Male_Reviews,Female_Reviews,Male_Score,Female_Score
0,0,7041,9.3,2666904,566032,1672983,1475251,294889,9.3,9.2
1,1,7042,9.2,1848055,417566,1161734,1072921,166441,9.2,8.9
2,2,7043,9.0,2639814,621027,1646769,1490958,267216,9.0,8.7
3,3,7044,9.0,1265636,282291,805848,759883,105690,9.0,8.7
4,4,7045,9.0,787589,187528,509017,453568,76860,9.0,8.9


Drop first column

In [16]:
score_df.pop("Unnamed: 0")

0        0
1        1
2        2
3        3
4        4
      ... 
245    245
246    246
247    247
248    248
249    249
Name: Unnamed: 0, Length: 250, dtype: int64

In [17]:
score_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Score_ID          250 non-null    int64  
 1   IMDb_Score        250 non-null    float64
 2   Audience_Reviews  250 non-null    int64  
 3   US_Users          250 non-null    int64  
 4   Non_US_Users      250 non-null    int64  
 5   Male_Reviews      250 non-null    int64  
 6   Female_Reviews    250 non-null    int64  
 7   Male_Score        250 non-null    float64
 8   Female_Score      250 non-null    float64
dtypes: float64(3), int64(6)
memory usage: 17.7 KB


In [ ]:
clean audience revies (ex. 250,000 + to 250K+)

### with cleaned up tables, I need to save them

#### Saving as CSV

In [ ]:
TBD